In [63]:
import requests 
import re as regex
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import base64
import base58
import uuid
import jupyter_util as util

## Enter Invite Url:

In [64]:
agent1 = {}
plugin = {}
endpoint = "http://localhost:8150/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent1['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent1['url'])

http://agent1.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiNzJkOGI4YzItOGI1MS00MGI0LThiMzQtNGY2Njc0OGUzOGQ1IiwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vYWdlbnQxLmxvY2FsaG9zdCIsICJsYWJlbCI6ICJNYWluIChhZG1pbikiLCAicmVjaXBpZW50S2V5cyI6IFsiODhKaU55UnlxeHFzOVRDaVJpWW9DUTh6NXFtcEI4cWpOWTRxVnVZRkpHSHAiXX0=


In [65]:
# Process invite url, delete white spaces
agent1['url'] = agent1['url'].replace(" ", "")
# Regex(substitution) to extract only the invite string from url
agent1['invite_string_b64'] = regex.sub(
           r".*(c\_i\=)", 
           r"", 
           agent1['url'])
# Decoding invite string using base64 decoder
agent1['invite_string'] = base64.b64decode(agent1['invite_string_b64'])
# Converting our invite json string into a dictionary 
agent1['invite'] = json.loads(agent1['invite_string'])

print("Decoded invitation:\n")
pp.pprint(agent1['invite'])

Decoded invitation:

{'@id': '72d8b8c2-8b51-40b4-8b34-4f66748e38d5',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Main (admin)',
 'recipientKeys': ['88JiNyRyqxqs9TCiRiYoCQ8z5qmpB8qjNY4qVuYFJGHp'],
 'serviceEndpoint': 'http://agent1.localhost'}


In [66]:
### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = \
    base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = \
    base58.b58encode(plugin['keypair'][1]).decode("ascii")

print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

Private key: 
 2JsTWLe2HzTwUko4VtiiwuoRqsdwUjVrrgmWrx1q8aMYoH2kvRbWbz6G3TRa9nri83F4FjexMDJBrtT1LgxKE9Sq

Public key: 
 FN3DTYxQzS9zuN56EE6g1nqbBf9hd97EjHygdv5xTo5b


**Sending a connection request to acapy**

In [67]:
uniqueId = str(uuid.uuid4())
# print(uniqueId)

# our request body
message = {
        "@id":  uniqueId,
        "~transport": {
          "return_route": "all"
        },
        "@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/request",
        "label": "Plugin",
        "connection": {
          "DID": plugin['did'],
          "DIDDoc": {
            "@context": "https://w3id.org/did/v1",
            "id": plugin['did'],
            "publicKey": [{
              "id": plugin['did'] + "#keys-1",
              "type": "Ed25519VerificationKey2018",
              "controller": plugin['did'],
              "publicKeyBase58": plugin['public_key_b58']
            }],
            "service": [{
              "id": plugin['did'] + ";indy",
              "type": "IndyAgent",
              "recipientKeys": plugin['public_key_b58'],
              "serviceEndpoint": ""
            }]
          }
        }
      }

**Encoding/packing the request**

In [68]:
# encoding it with aries crypto function using the key that was
# given to us by aca-py in recipientKeys
decodedAcapyKey = base58.b58decode(agent1['invite']['recipientKeys'][0])
ourPrivateKey = plugin['keypair'][1]
# print(plugin['did'])
encodedMessage = \
    crypto.encode_pack_message(json.dumps(message), [decodedAcapyKey], ourPrivateKey)

encodedMessage = encodedMessage.decode("ascii")

In [69]:
connectionRequestResponse = requests.post(agent1['invite']['serviceEndpoint'], data=encodedMessage)
assert connectionRequestResponse.text != "", "invalid response from acapy"

**Response from aca-py**

In [70]:
   
connectionRequestResponseUnpacked = \
    util.unpackMessage(connectionRequestResponse.text, plugin['keypair'][1])

connectionRequestResponseDict = json.loads(connectionRequestResponseUnpacked[0])

print("Decoded acapy response: \n")
pp.pprint(connectionRequestResponseUnpacked)

Decoded acapy response: 

('{"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/response", '
 '"@id": "99661ec5-6c78-40d2-8d78-edc677aff7f1", "~thread": {"thid": '
 '"5b3ecc6a-be48-4f5d-b94f-368e1d34be26"}, "connection~sig": {"@type": '
 '"did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/signature/1.0/ed25519Sha512_single", '
 '"signature": '
 '"Cw8AB65mRDXexFU2xgU0yoYs0jXcRiTEZeyBIJkRIrwNTIDSx4tDvq-lMouxwBwIwjjs2eNN6PFa36F7A0iACA==", '
 '"sig_data": '
 '"AAAAAF7E_OB7IkRJRCI6ICJOSGRiS05IVEpzZjlGZnQ1Q3llQ0F0IiwgIkRJRERvYyI6IHsiQGNvbnRleHQiOiAiaHR0cHM6Ly93M2lkLm9yZy9kaWQvdjEiLCAiaWQiOiAiZGlkOnNvdjpOSGRiS05IVEpzZjlGZnQ1Q3llQ0F0IiwgInB1YmxpY0tleSI6IFt7ImlkIjogImRpZDpzb3Y6TkhkYktOSFRKc2Y5RmZ0NUN5ZUNBdCMxIiwgInR5cGUiOiAiRWQyNTUxOVZlcmlmaWNhdGlvbktleTIwMTgiLCAiY29udHJvbGxlciI6ICJkaWQ6c292Ok5IZGJLTkhUSnNmOUZmdDVDeWVDQXQiLCAicHVibGljS2V5QmFzZTU4IjogIkNidjdrWWhnNVAyb2p0dnBjWVR5S2NYWmZONFFSV25kTjVuOVhxb3liamNmIn1dLCAiYXV0aGVudGljYXRpb24iOiBbeyJ0eXBlIjogIkVkMjU1MTlTaWduYXR1cmVBdXRoZW50aWNhdGlvbjIwMT

**Unpacking connection data embedded in the response**

In [71]:
sig_data_raw = connectionRequestResponseDict['connection~sig']['sig_data']
#  (this is a hack)replacing artifacts that sometimes happen
sig_data_raw = sig_data_raw.replace("-", "1")
sig_data_raw = sig_data_raw.replace("_", "1")

sig_data_raw = base64.b64decode(sig_data_raw)
# avoid first 8 characters as they are a time signature
sig_data_raw = sig_data_raw[8:]
sig_data_raw = sig_data_raw.decode('ascii')

sig_data = json.loads(sig_data_raw)
pp.pprint(sig_data)

{'DID': 'NHdbKNHTJsf9Fft5CyeCAt',
 'DIDDoc': {'@context': 'https://w3id.org/did/v1',
            'authentication': [{'publicKey': 'did:sov:NHdbKNHTJsf9Fft5CyeCAt#1',
                                'type': 'Ed25519SignatureAuthentication2018'}],
            'id': 'did:sov:NHdbKNHTJsf9Fft5CyeCAt',
            'publicKey': [{'controller': 'did:sov:NHdbKNHTJsf9Fft5CyeCAt',
                           'id': 'did:sov:NHdbKNHTJsf9Fft5CyeCAt#1',
                           'publicKeyBase58': 'Cbv7kYhg5P2ojtvpcYTyKcXZfN4QRWndN5n9Xqoybjcf',
                           'type': 'Ed25519VerificationKey2018'}],
            'service': [{'id': 'did:sov:NHdbKNHTJsf9Fft5CyeCAt;indy',
                         'priority': 0,
                         'recipientKeys': ['Cbv7kYhg5P2ojtvpcYTyKcXZfN4QRWndN5n9Xqoybjcf'],
                         'serviceEndpoint': 'http://agent1.localhost',
                         'type': 'IndyAgent'}]}}


**Adding the connection to connection list**

In [72]:
connections = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": sig_data['DIDDoc'],
        "myKey": plugin['keypair'][1]
    }
]

pp.pprint(connections)

[{'DIDDoc': {'@context': 'https://w3id.org/did/v1',
             'authentication': [{'publicKey': 'did:sov:NHdbKNHTJsf9Fft5CyeCAt#1',
                                 'type': 'Ed25519SignatureAuthentication2018'}],
             'id': 'did:sov:NHdbKNHTJsf9Fft5CyeCAt',
             'publicKey': [{'controller': 'did:sov:NHdbKNHTJsf9Fft5CyeCAt',
                            'id': 'did:sov:NHdbKNHTJsf9Fft5CyeCAt#1',
                            'publicKeyBase58': 'Cbv7kYhg5P2ojtvpcYTyKcXZfN4QRWndN5n9Xqoybjcf',
                            'type': 'Ed25519VerificationKey2018'}],
             'service': [{'id': 'did:sov:NHdbKNHTJsf9Fft5CyeCAt;indy',
                          'priority': 0,
                          'recipientKeys': ['Cbv7kYhg5P2ojtvpcYTyKcXZfN4QRWndN5n9Xqoybjcf'],
                          'serviceEndpoint': 'http://agent1.localhost',
                          'type': 'IndyAgent'}]},
  'label': 'Main (admin)',
  'myKey': b'Amg\xb0DQ\x9d\xc6\xbacMW\x8cGq\xc5\xfb\xf5\x05IEtP\x95\x

### FEATURE DISCOVERY

In [73]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/query",
    query="*"
)
message = util.packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=message)
# print(response.text)

responseDecoded = util.unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': '4775d17d-f4a6-4642-9195-3647b783c7d0',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/disclose',
 'protocols': [{'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/credential-issuance/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/routing/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/action-menu/1.0',
                'roles': ['provider']},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0'},
               {'pid': 'https://didcomm.org/introduction-service/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/credential-presentation/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/basicmessage/1.0'},
               {'pid': 'https://didcomm.org/connections/1.0'},
               {'pid': 'https://didcomm

In [76]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/schema",
    payload="{amazing: 'payload'}"
)

encodedMessage = util.packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = util.unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': '30c547cb-d33b-47da-af4a-205775eed071',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/schema',
 'payload': "{amazing: 'payload'}",
 'schema_id': '2a272d933be14d9e9a92589395ed75db',
 '~thread': {'thid': '5b2b3339-feaa-4fe7-8f6c-f49fe356ca16'}}


In [78]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/search_schema",
    schema_id="2a272d933be14d9e9a92589395ed75db"
)

encodedMessage = util.packMessage(message, connections[0])

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)

responseDecoded = util.unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': 'd7e34e17-fb37-4a30-8a5d-3d13ac660a8f',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/schema',
 'payload': "{amazing: 'payload'}",
 'schema_id': '2a272d933be14d9e9a92589395ed75db',
 '~thread': {'thid': '3f56f12d-271d-4d4a-bb96-304eb9ad4da1'}}
